#PyTorch 튜토리얼 

Tensor(텐서)는 Numpy의 다차원 행렬 자료구조인 ndarray와 유사하나 CPU 뿐만 아니라 GPU에서 실행될 수 있다는 차이점이 있습니다.

PyTorch는 GPU와 CPU를 사용하는 딥러닝을 위한 최적화된 텐서 라이브러리입니다.

본 코드에서 PyTorch를 사용하여 CNN, RNN 모델을 구현할 때 알아야할 기본적인 사항을 소개합니다. 

코드 출처: https://tutorials.pytorch.kr/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py

PyTorch Documents: https://pytorch.org/docs/stable/index.html

In [2]:
# The torch package contains data structures for multi-dimensional tensors and 
# defines mathematical operations over these tensors. 
import torch
import numpy as np

텐서 초기화하기

In [3]:
# 2차원 리스트 데이터로부터 텐서 생성
data = [[1, 2], [3, 4]]
# 텐서 생성
x_data = torch.tensor(data)
print(x_data)
print(x_data.shape)

tensor([[1, 2],
        [3, 4]])
torch.Size([2, 2])


In [4]:
# 랜덤값으로 텐서 생성. 정수값으로 텐서의 차원을 전달
# 랜덤값으로 채워진 (3, 4) 차원의 텐서를 생성 
tensor = torch.rand(3, 4)   
print ("tensor :", tensor)
print(f"Shape of tensor: {tensor.shape}")     
print(f"Datatype of tensor: {tensor.dtype}") 
print(f"Device tensor is stored on: {tensor.device}") 

tensor : tensor([[9.1212e-01, 5.6359e-01, 4.5931e-04, 3.2841e-01],
        [8.6146e-01, 5.8954e-01, 7.1581e-01, 3.0010e-01],
        [4.4201e-01, 2.0217e-02, 6.2258e-01, 1.5160e-01]])
Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [5]:
# Returns a tensor filled with the scalar value 1,with the shape defined by the argument size.
tensor = torch.ones(4, 4)
print(tensor)
# 모든 행에 대해서 1번열에 0값을 대입
tensor[:,1] = 0
print(tensor)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [6]:
# Concatenate : 텐서를 연결하기
# 딥러닝에서는 모델의 입력 또는 중간 연산 단계에서 두 개의 텐서를 연결하는 경우가 있습니다.
# 두 텐서를 연결해서 입력으로 사용하는 것은 두 텐서에 담긴 정보를 모두 사용한다는 의미입니다. 
# dim : 텐서를 연결하여 어느 차원을 늘릴 것인지를 표시
t1 = torch.cat([tensor, tensor], dim=0) # 두 텐서를 연결하여 0번째 차원을 늘리라는 의미
print("tensor shape:", tensor.shape)
print("->", t1)
print("t1 shape:", t1.shape) # 0번째 dimension이 늘어난 것을 확인
print("----------------------------")
t2 = torch.cat([tensor, tensor], dim=1) # 두 텐서를 연결하여 1번째 차원을 늘리라는 의미
print("tensor shape:", tensor.shape)
print("->", t2)
print("t2 shape:", t2.shape) # 1번째 dimension이 늘어난 것을 확인

tensor shape: torch.Size([4, 4])
-> tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
t1 shape: torch.Size([8, 4])
----------------------------
tensor shape: torch.Size([4, 4])
-> tensor([[1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1.]])
t2 shape: torch.Size([4, 8])


#신경망 모델 생성하기

모든 신경망 클래스는 torch.nn 패키지를 통해서 생성합니다.

torch.nn.Module은 PyTorch의 모든 신경망의 Base Class이며 새로운 신경망 모델은 torch.nn.Module 클래스를 상속하여 정의해야 합니다.

새로운 클래스 내에서 \__init()__함수와 forward()함수를 반드시 override 해야 합니다.

\__init()__함수에서는 모델에서 사용될 module(nn.Linear, nn.Conv2d), activation function(ReLU 등)를 정의합니다. 

forward()에서는 모델에서 실행되어야 하는 연산을 정의합니다.

backward 연산은 backward() 함수를 호출하면 PyTorch가 자동으로 수행하므로 forward()만 정의합니다.

forward()에서는 input 데이터에 대해 어떤 연산을 진행하여 output이 나올지를 정의해 주는 것입니다. 

In [7]:
import torch.nn as nn           # 신경망 구현을 위한 데이터 구조, 신경망 레이어 등이 정의되어 있음 
import torch.nn.functional as F # Convolution, Pooling, Activation, Linear 함수 등이 정의되어 있음

In [8]:
# torch.nn.Module은 PyTorch의 모든 신경망의 Base Class
# __init()__과 forward()를 반드시 override 해야 한다.
class Net(nn.Module):
  def __init__(self): # init과 foward 함수 꼭 재정의를 해주어야 함
    super(Net, self).__init__()
    # 입력 이미지 채널 1개, 출력 채널 6개, 5x5의 정사각 컨볼루션 행렬
    # 컨볼루션 커널 정의
    # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1)
    self.conv1 = nn.Conv2d(1, 6, 5)   # 입력 채널 크기, 출력 채널 크기, 커널 크기
    self.conv2 = nn.Conv2d(6, 16, 5)  # 입력 채널 크기, 출력 채널 크기, 커널 크기
    
    # Fully Connected Layer: y = Wx + b
    self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5은 이미지 차원에 해당
    self.fc2 = nn.Linear(120, 60) # 120개의 차원을 60개의 차원으로 계산
    self.fc3 = nn.Linear(60, 10)  # 60 -> 10개 차원으로 계산

  def forward(self, x):
    # convolution을 거치게 되면 이미지의 크기는 kernel - 1만큼 감소함
    # 현재 입력되는 이미지의 크기가 32 X 32
    # conv1을 통해 출력되는 이미지의 크기는 (32 - 5 + 1) X (32 - 5 + 1) = 28 X 28
    # (2, 2) 크기 윈도우에 대해 맥스 풀링 -> 이미지의 크기는 2분의 1이 되므로 -> 최종 출력 이미지 크기는 14 X 14
    # torch.nn.functional.max_pool2d(input, kernel_size)
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # [batch size, 채널 크기, 이미지 가로 크기, 이미지 세로 크기] == [1, 6, 14, 14] 

    # conv2을 통해 출력되는 이미지의 크기는 (14 - 5 + 1) X (14 - 5 + 1) = 10 X 10
    # (2, 2) 크기 윈도우에 대해 맥스 풀링 -> 이미지의 크기는 2분의 1이 되므로 -> 최종 출력 이미지 크기는 5 X 5
    # 크기가 제곱수라면, 하나의 숫자만을 특정(specify)
    x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2)) # [1, 16, 5, 5]
    # torch.flatten(input, start_dim, end_dim) : flattens input by reshaping it into a one-dimensional tensor. 
    x = torch.flatten(x, 1) # (tensor, dimension) batch 차원을 제외한 모든 차원을 하나로 평탄화(flatten) [1, 16 * 5 * 5] 들어가는 input의 차원은 16*5*5 = 400
    x = F.relu(self.fc1(x)) # [1, 120]
    x = F.relu(self.fc2(x)) # [1, 60]
    x = self.fc3(x) # [1, 10]
    return x

In [9]:
net = Net()
print(net) # 신경망 모델의 구조

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)


In [10]:
input = torch.randn(1, 1, 32, 32) # [batch size, 입력 채널 크기, 가로 길이, 세로 길이] -> 입력데이터
out = net(input)
print("out shape:", out.shape)
print(out)  # out shape: (1, 10)

out shape: torch.Size([1, 10])
tensor([[-0.0232, -0.0278, -0.0129,  0.0035,  0.0384, -0.0901,  0.0932,  0.0244,
         -0.1483, -0.1132]], grad_fn=<AddmmBackward0>)


In [11]:
# view 함수 : reshape 함수. 텐서의 형태(Shape)를 변경함. 변경 전과 후에 원소의 갯수는 유지되어야 한다.
# view(1, -1) : 첫번째 차원은 1이 되도록 하되,-1로 표시된 2번째 차원은 파이토치가 알아서 계산하라는 의미
output = net(input)               # 입력 데이터를 신경망 모델에 전달하여 예측값을 얻음 shape: (1, 10)
print("output shape:", output.shape)
target = torch.randn(10)          # 임의의 텐서를 생성하여 정답값으로 가정
print ("target shape:", target.shape)
target = target.view(1, -1)       # 모델의 출력 텐서와 동일한 shape로 변경 : (1, 10)
                                  # 정답값으로 가정한 target도 output 차원과 동일하게 변경 view(1, -1) => 0번째는 1주고 1번째는 알맞게 넣어라
print ("after reshape(view) -> target shape:", target.shape)


output shape: torch.Size([1, 10])
target shape: torch.Size([10])
after reshape(view) -> target shape: torch.Size([1, 10])


손실함수 (Loss Function) 설정

In [12]:
import torch
import torch.nn as nn
# 예측값과 정답값 사이의 오차를 줄이면서 최적화해 나감
# 어떤 텐서가 학습에 필요한 텐서라면 backpropagation을 통하여 gradient를 구해야 합니다.
# 텐서의 옵션 requireds_grad를 True로 설정하면 텐서에 실행되는 모든 연산들을 트랙킹하여 자동으로 gradient를 계산합니다.
input = torch.randn(3, 5, requires_grad=True) 
target = torch.randn(3, 5)
 # 손실함수로 MSE(Mean Squared Error)를 설정
 # 평균제곱오차(MSE)는 오차를 제곱한 값의 평균. 오차란 모델이 예측한 값과 실제 정답과의 차이
criterion = nn.MSELoss()          
output = criterion(input, target) # Loss 계산
output.backward() # 역전파 진행
print('input: ', input)
print('target: ', target)
print('output: ', output) # 임의의 실수값을 가진 tensor

# 손실함수 설정 시 nn에서 어떤 함수를 불러와서 loss를 계산하는 상황

input:  tensor([[ 0.4468, -0.6449,  0.7325,  1.3127, -0.1208],
        [ 1.1454,  0.2387, -0.6937, -0.0083,  1.1390],
        [-0.6047,  0.2860, -0.8666, -0.4307,  1.9010]], requires_grad=True)
target:  tensor([[-0.4074,  0.2900, -0.8044,  0.7536, -0.8668],
        [-0.5720, -0.9828,  0.7305,  2.5481, -1.3244],
        [-1.1969,  0.3493,  0.8787,  0.2553, -0.2976]])
output:  tensor(2.1742, grad_fn=<MseLossBackward0>)


옵티마이저 설정

In [13]:
# torch.optim : 신경망 학습을 위한 파라미터 최적화 알고리즘이 구현되어 있는 클래스
import torch.optim as optim
# Adam Optimizer 객체 생성
optimizer = optim.Adam(net.parameters(), lr=0.01) # adam 옵티마이저 객체 생성 : 학습가능한 파라미터들을 넘겨줌(net 우리가 정의한 신경망을 담고 있음), 학습률

# 학습 과정(training loop)
# Pytorch에서는 gradients 값들을 backward를 할 때 계속 누적하기 때문에 
# 학습을 시작하기 전에 gradients 버퍼를 zero로 reset해야 한다.
optimizer.zero_grad()

input = torch.randn(1, 1, 32, 32)
output = net(input)               # 이 클래스에 input 전달한 순간에 foward 실행됨 => conV 등등 연산이 진행
target = torch.randn(10)          # 예시를 위한 임의의 정답
target = target.view(1, -1)       # 출력과 같은 shape로 만듬
loss = criterion(output, target)  # Loss 계산
print(loss)

loss.backward()         # 역전파 함수 실행을 통해 gradient 계산
optimizer.step()        # gradient를 기반으로 실제 파라미터 업데이트를 실행

tensor(1.8221, grad_fn=<MseLossBackward0>)


In [14]:
# torch.optim : 신경망 학습을 위한 파라미터 최적화 알고리즘들이 구현되어 있는 팩키지
import torch.optim as optim
# Optimizer 객체를 생성하고 모델의 파라미터를 전달, learning rate 설정
optimizer = optim.Adam(net.parameters(), lr=0.01) 

# 학습 과정(training loop)
# Pytorch에서는 gradients 값들을 backward를 할 때 계속 누적하기 때문에 
# 학습을 시작하기 전에 gradients 버퍼를 zero로 reset해야 한다.
optimizer.zero_grad()

input = torch.randn(1, 1, 32, 32)
output = net(input)
target = torch.randn(10)          # 예시를 위한 임의의 정답
target = target.view(1, -1)       # 출력과 같은 shape로 만듬
loss = criterion(output, target)  # Loss 계산
print(loss)

loss.backward()         # 역전파 함수 실행을 통해 gradient 계산
optimizer.step()        # 파라미터 업데이트를 실행

tensor(0.9062, grad_fn=<MseLossBackward0>)


GPU 가속 이용하기

In [15]:
# 현재 개발환경에서 GPU 가속이 가능한지 확인
print(torch.cuda.is_available()) #  True => GPU를 사용할 수 있는 환경

True


In [16]:
# GPU 사용이 가능하면 cuda로 연산하도록 device를 설정 그렇지 않으면 cpu로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device) # 이용가능한 device에서 적용

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)

PyTorch TensorDataset, DataLoader, RandomSampler, SequentialSampler

데이터셋을 샘플링하고, 학습 루프를 편리하게 구현하기 위한 함수

In [17]:
# 랜덤한 학습 데이터 생성
train_inputs = torch.randn(100, 32, 128) # [전체 데이터 개수, 데이터의 최대 길이, 히든 벡터 크기]
train_labels = torch.randn(100, 3) # [전체 데이터 개수, 예측할 클래스 개수]


In [18]:
# Dataset에 필요한 패키지 임포트
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [21]:
batch_size = 12 # 매 학습 Step 마다 샘플링할 데이터 개수

train_data = TensorDataset(train_inputs, train_labels) # 학습데이터와 레이블을 하나의 TensorDataset으로 결합 가능
train_sampler = RandomSampler(train_data) # 데이터를 샘플링 할 함수(순차적으로 뽑아올지, 랜덤하게 뽑아올지) => 데이터를 학습 시 랜덤하게 하는 것
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size) # 학습 시 미니배치 단위로 데이터를 자동 로딩 

In [22]:
# dataloader를 통해서 학습 루프 생성
for step, batch in enumerate(train_dataloader): # enumerate : 인덱스(index)와 데이터값에 동시에 접근 / batch 사이즈 만큼 배치에 데이터를 로드
  x_data, x_label = batch
  # 데이터 로딩 step 
  print(step, x_data.shape, x_label.shape)

0 torch.Size([12, 32, 128]) torch.Size([12, 3])
1 torch.Size([12, 32, 128]) torch.Size([12, 3])
2 torch.Size([12, 32, 128]) torch.Size([12, 3])
3 torch.Size([12, 32, 128]) torch.Size([12, 3])
4 torch.Size([12, 32, 128]) torch.Size([12, 3])
5 torch.Size([12, 32, 128]) torch.Size([12, 3])
6 torch.Size([12, 32, 128]) torch.Size([12, 3])
7 torch.Size([12, 32, 128]) torch.Size([12, 3])
8 torch.Size([4, 32, 128]) torch.Size([4, 3])


In [ ]:
# 마지막 9번째는 배치사이즈 12개를 못채워서 4개만